In [1]:
# preprocess_clip_embeddings.py

import os
import torch
import clip
from PIL import Image
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# 配置参数
IMG_DIR = '/root/autodl-tmp/celeba_datasets/img_align_celeba/img_align_celeba'
ATTR_PATH = '/root/autodl-tmp/celeba_datasets/list_attr_celeba.txt'
BBOX_PATH = '/root/autodl-tmp/celeba_datasets/list_bbox_celeba.txt'
PARTITION_PATH = '/root/autodl-tmp/celeba_datasets/list_eval_partition.txt'

OUTPUT_DIR = './clip_embeddings'  # 输出目录
os.makedirs(OUTPUT_DIR, exist_ok=True)

IMAGE_SIZE = 224  # CLIP ViT-B/32的默认图像尺寸
BATCH_SIZE = 512  # 根据GPU显存调整
NUM_WORKERS = 4  # 根据CPU核心数调整

# 设备设置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# 图像预处理
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.48145466, 0.4578275, 0.40821073], 
                         [0.26862954, 0.26130258, 0.27577711])  # CLIP的归一化参数
])

# 数据集类
class CelebAEmbeddingsDataset(Dataset):
    def __init__(self, img_dir, attr_path, bbox_path, partition_path, transform=None, partition=0):
        self.img_dir = img_dir
        self.transform = transform

        # 读取属性文件
        try:
            attr_df = pd.read_csv(attr_path, sep=',', header=0)  # 正确设置 header=0
        except Exception as e:
            print(f"Error reading {attr_path}: {e}")
            raise

        print("Attributes DataFrame columns:", attr_df.columns.tolist())
        print("Attributes DataFrame head:\n", attr_df.head())

        # 确认 'image_id' 列存在
        if 'image_id' not in attr_df.columns:
            raise ValueError("属性文件中缺少 'image_id' 列。请检查文件格式。")
        else:
            print("'image_id' column found in attributes DataFrame.")

        # 读取分区文件
        try:
            partition_df = pd.read_csv(partition_path, sep=',', header=0, names=['image_id', 'partition'])
        except Exception as e:
            print(f"Error reading {partition_path}: {e}")
            raise

        print("Partition DataFrame columns:", partition_df.columns.tolist())
        print("Partition DataFrame head:\n", partition_df.head())

        # 读取边界框文件
        try:
            bbox_df = pd.read_csv(bbox_path, sep=',', header=0, names=['image_id', 'x_1', 'y_1', 'width', 'height'])
        except Exception as e:
            print(f"Error reading {bbox_path}: {e}")
            raise

        print("Bounding Box DataFrame columns:", bbox_df.columns.tolist())
        print("Bounding Box DataFrame head:\n", bbox_df.head())

        # 合并属性和分区数据
        try:
            attr_df = attr_df.merge(partition_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes and partitions: {e}")
            raise

        print("After merging with partition, columns:", attr_df.columns.tolist())
        print("Merged DataFrame head:\n", attr_df.head())

        # 合并属性、分区和边界框数据
        try:
            attr_df = attr_df.merge(bbox_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes, partitions and bounding boxes: {e}")
            raise

        print("After merging with bounding box, columns:", attr_df.columns.tolist())
        print("Final Merged DataFrame head:\n", attr_df.head())

        # 筛选分区
        print(f"Total samples before filtering: {len(attr_df)}")
        self.attr_df = attr_df[attr_df['partition'] == partition].reset_index(drop=True)
        print(f"Total samples after filtering partition {partition}: {len(self.attr_df)}")

        if len(self.attr_df) == 0:
            raise ValueError(f"训练集中没有样本。请检查数据文件的读取和过滤逻辑。")

        # 获取属性名称
        # 属性列是除了 'image_id', 'partition', 'x_1', 'y_1', 'width', 'height' 之外的所有列
        self.attr_names = [col for col in attr_df.columns if col not in ['image_id', 'partition', 'x_1', 'y_1', 'width', 'height']]
        print("Attribute names:", self.attr_names)

    def __len__(self):
        return len(self.attr_df)

    def __getitem__(self, idx):
        # 获取图像路径
        img_name = self.attr_df.iloc[idx, self.attr_df.columns.get_loc('image_id')]
        img_path = os.path.join(self.img_dir, img_name)
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error opening image {img_path}: {e}")
            # 可以选择跳过该样本，或返回一个默认图像
            # 这里选择抛出异常
            raise

        if self.transform:
            image = self.transform(image)

        # 获取属性标签并生成文本描述
        attrs = self.attr_df.iloc[idx][self.attr_names].values
        attrs = (attrs + 1) // 2  # 将-1转为0，1保持1
        attrs = attrs.astype(int)
        active_attrs = [attr for attr, val in zip(self.attr_names, attrs) if val == 1]

        # 手动截断属性数量，例如，最多10个属性，以避免超长文本
        max_attrs = 10
        active_attrs = active_attrs[:max_attrs]

        if len(active_attrs) == 0:
            text_prompt = "A person with no notable attributes."
        else:
            text_description = ', '.join(active_attrs)
            text_prompt = f"A person with {text_description}."

        # 打印前5个样本的文本描述和图像形状
        if idx < 5:
            print(f"Index: {idx}, Text Prompt: {text_prompt}")
            print(f"Image shape: {image.shape}")

        return text_prompt, image

# 初始化 CLIP 模型
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model.eval()

# 创建数据集和数据加载器
try:
    dataset = CelebAEmbeddingsDataset(
        img_dir=IMG_DIR,
        attr_path=ATTR_PATH,
        bbox_path=BBOX_PATH,
        partition_path=PARTITION_PATH,
        transform=transform,
        partition=0  # 0: train
    )
except Exception as e:
    print(f"Error initializing dataset: {e}")
    raise

if len(dataset) == 0:
    raise ValueError("训练集中没有样本。请检查数据文件的读取和过滤逻辑。")

dataloader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS,
    pin_memory=True
)

# 计算并保存嵌入
text_embeddings = []
image_embeddings = []

with torch.no_grad():
    for text_prompts, images in tqdm(dataloader, desc="Computing Embeddings"):
        # 生成文本嵌入
        try:
            text_tokens = clip.tokenize(text_prompts, truncate=True).to(device)
        except Exception as e:
            print(f"Error tokenizing text: {e}")
            raise

        try:
            text_emb = clip_model.encode_text(text_tokens).cpu()
        except Exception as e:
            print(f"Error encoding text: {e}")
            raise

        text_embeddings.append(text_emb)

        # 生成图像嵌入
        try:
            images = images.to(device)
            image_emb = clip_model.encode_image(images).cpu()
        except Exception as e:
            print(f"Error encoding images: {e}")
            raise

        image_embeddings.append(image_emb)

# 合并所有嵌入
if len(text_embeddings) == 0 or len(image_embeddings) == 0:
    raise RuntimeError("未计算到任何嵌入。请检查数据加载和嵌入计算步骤。")

text_embeddings = torch.cat(text_embeddings, dim=0)
image_embeddings = torch.cat(image_embeddings, dim=0)

# 保存嵌入到文件
torch.save(text_embeddings, os.path.join(OUTPUT_DIR, 'text_embeddings.pt'))
torch.save(image_embeddings, os.path.join(OUTPUT_DIR, 'image_embeddings.pt'))

print("嵌入已保存为 'text_embeddings.pt' 和 'image_embeddings.pt'")


使用设备: cuda
Attributes DataFrame columns: ['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']
Attributes DataFrame head:
      image_id  5_o_Clock_Shadow  Arched_Eyebrows  Attractive  Bags_Under_Eyes  \
0  000001.jpg                -1                1           1               -1   
1  000002.jpg                -1               -1          -1                1   
2  000003.jpg                -1               -1          -1               -1   
3  000004.

Computing Embeddings:   0%|          | 0/318 [00:00<?, ?it/s]

Index: 0, Text Prompt: A person with Arched_Eyebrows, Attractive, Brown_Hair, Heavy_Makeup, High_Cheekbones, Mouth_Slightly_Open, No_Beard, Pointy_Nose, Smiling, Straight_Hair.
Image shape: torch.Size([3, 224, 224])
Index: 1, Text Prompt: A person with Bags_Under_Eyes, Big_Nose, Brown_Hair, High_Cheekbones, Mouth_Slightly_Open, No_Beard, Smiling, Young.
Image shape: torch.Size([3, 224, 224])
Index: 2, Text Prompt: A person with Big_Lips, Blurry, Male, Narrow_Eyes, No_Beard, Pointy_Nose, Wavy_Hair, Young.
Image shape: torch.Size([3, 224, 224])
Index: 3, Text Prompt: A person with Attractive, No_Beard, Pointy_Nose, Straight_Hair, Wearing_Earrings, Wearing_Lipstick, Wearing_Necklace, Young.
Image shape: torch.Size([3, 224, 224])
Index: 4, Text Prompt: A person with Arched_Eyebrows, Attractive, Big_Lips, Heavy_Makeup, Narrow_Eyes, No_Beard, Pointy_Nose, Wearing_Lipstick, Young.
Image shape: torch.Size([3, 224, 224])


Computing Embeddings: 100%|██████████| 318/318 [02:15<00:00,  2.34it/s]


嵌入已保存为 'text_embeddings.pt' 和 'image_embeddings.pt'


In [2]:
import torch

text_emb_sample = torch.load('./clip_embeddings/text_embeddings.pt')
image_emb_sample = torch.load('./clip_embeddings/image_embeddings.pt')

print(f"文本嵌入样本形状: {text_emb_sample.shape}")  # 应为 [N, 512]
print(f"图像嵌入样本形状: {image_emb_sample.shape}")  # 应为 [N, 512]

# 打印第一个样本的嵌入
print("第一个文本嵌入:", text_emb_sample[0])
print("第一个图像嵌入:", image_emb_sample[0])


/tmp/ipykernel_4722/3045327233.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  text_emb_sample = torch.load('./clip_embeddings/text_embeddings.pt')
/tmp/ipykernel_4722/3

文本嵌入样本形状: torch.Size([162770, 512])
图像嵌入样本形状: torch.Size([162770, 512])
第一个文本嵌入: tensor([ 4.1901e-02,  3.7549e-01, -2.2180e-01,  1.8335e-01,  8.2458e-02,
        -2.4121e-01, -2.4866e-01,  7.4268e-01,  7.8796e-02,  4.9707e-01,
         1.7554e-01, -4.1534e-02,  3.5938e-01,  7.9163e-02,  2.6709e-01,
         1.6577e-01,  5.2295e-01,  1.6785e-01, -2.5879e-01,  2.2693e-01,
         4.9487e-01,  1.0559e-01, -2.4988e-01, -5.7434e-02, -1.7017e-01,
        -4.9408e-02, -2.0850e-01,  1.2494e-01, -3.4497e-01, -1.4478e-01,
         2.2058e-01, -2.9736e-01, -1.3196e-01,  1.3403e-01,  2.2412e-01,
         1.3123e-01,  3.3997e-02,  7.4951e-02, -1.2549e-01,  2.5024e-01,
        -1.8457e-01, -9.9121e-02, -1.6321e-01,  8.6365e-02, -1.2854e-01,
         5.7520e-01,  1.4392e-01, -7.4524e-02, -9.7961e-02,  1.4917e-01,
         8.5388e-02, -2.1866e-02,  1.6492e-01, -3.1714e-01, -9.6130e-02,
        -1.4075e-01,  2.3010e-01,  5.4504e-02, -3.0914e-02,  1.6650e-01,
         5.7520e-01, -2.0178e-01,  1.3928e-

## 下面这个是新的嵌入生成，是我们为了训练和测试过程都用图像嵌入去运行代码（这样效果一定很好，但是违规哟，测试用文本才可以）

In [3]:
# preprocess_clip_embeddings.py

import os
import torch
import clip
from PIL import Image
import pandas as pd
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

# 配置参数
IMG_DIR = '/root/autodl-tmp/celeba_datasets/img_align_celeba/img_align_celeba'
ATTR_PATH = '/root/autodl-tmp/celeba_datasets/list_attr_celeba.txt'
BBOX_PATH = '/root/autodl-tmp/celeba_datasets/list_bbox_celeba.txt'
PARTITION_PATH = '/root/autodl-tmp/celeba_datasets/list_eval_partition.txt'

OUTPUT_DIR = './clip_embeddings'  # 输出目录
os.makedirs(OUTPUT_DIR, exist_ok=True)

IMAGE_SIZE = 224  # CLIP ViT-B/32的默认图像尺寸
BATCH_SIZE = 512  # 根据GPU显存调整
NUM_WORKERS = 4  # 根据CPU核心数调整

# 设备设置
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")

# 图像预处理
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.48145466, 0.4578275, 0.40821073], 
                         [0.26862954, 0.26130258, 0.27577711])  # CLIP的归一化参数
])

# 数据集类
class CelebAEmbeddingsDataset(Dataset):
    def __init__(self, img_dir, attr_path, bbox_path, partition_path, transform=None, partition=0):
        self.img_dir = img_dir
        self.transform = transform

        # 读取属性文件
        try:
            attr_df = pd.read_csv(attr_path, sep=',', header=0)
        except Exception as e:
            print(f"Error reading {attr_path}: {e}")
            raise

        print("Attributes DataFrame columns:", attr_df.columns.tolist())
        print("Attributes DataFrame head:\n", attr_df.head())

        # 确认 'image_id' 列存在
        if 'image_id' not in attr_df.columns:
            raise ValueError("属性文件中缺少 'image_id' 列。请检查文件格式。")
        else:
            print("'image_id' column found in attributes DataFrame.")

        # 读取分区文件
        try:
            partition_df = pd.read_csv(partition_path, sep=',', header=0, names=['image_id', 'partition'])
        except Exception as e:
            print(f"Error reading {partition_path}: {e}")
            raise

        print("Partition DataFrame columns:", partition_df.columns.tolist())
        print("Partition DataFrame head:\n", partition_df.head())

        # 读取边界框文件
        try:
            bbox_df = pd.read_csv(bbox_path, sep=',', header=0, names=['image_id', 'x_1', 'y_1', 'width', 'height'])
        except Exception as e:
            print(f"Error reading {bbox_path}: {e}")
            raise

        print("Bounding Box DataFrame columns:", bbox_df.columns.tolist())
        print("Bounding Box DataFrame head:\n", bbox_df.head())

        # 合并属性和分区数据
        try:
            attr_df = attr_df.merge(partition_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes and partitions: {e}")
            raise

        print("After merging with partition, columns:", attr_df.columns.tolist())
        print("Merged DataFrame head:\n", attr_df.head())

        # 合并属性、分区和边界框数据
        try:
            attr_df = attr_df.merge(bbox_df, on='image_id')
        except Exception as e:
            print(f"Error merging attributes, partitions and bounding boxes: {e}")
            raise

        print("After merging with bounding box, columns:", attr_df.columns.tolist())
        print("Final Merged DataFrame head:\n", attr_df.head())

        # 筛选分区
        print(f"Total samples before filtering: {len(attr_df)}")
        self.attr_df = attr_df[attr_df['partition'] == partition].reset_index(drop=True)
        print(f"Total samples after filtering partition {partition}: {len(self.attr_df)}")

        if len(self.attr_df) == 0:
            raise ValueError(f"训练集中没有样本。请检查数据文件的读取和过滤逻辑。")

        # 获取属性名称
        # 属性列是除了 'image_id', 'partition', 'x_1', 'y_1', 'width', 'height' 之外的所有列
        self.attr_names = [col for col in attr_df.columns if col not in ['image_id', 'partition', 'x_1', 'y_1', 'width', 'height']]
        print("Attribute names:", self.attr_names)

    def __len__(self):
        return len(self.attr_df)

    def __getitem__(self, idx):
        # 获取图像路径
        img_name = self.attr_df.iloc[idx, self.attr_df.columns.get_loc('image_id')]
        img_path = os.path.join(self.img_dir, img_name)
        try:
            image = Image.open(img_path).convert('RGB')
        except Exception as e:
            print(f"Error opening image {img_path}: {e}")
            # 可以选择跳过该样本，或返回一个默认图像
            # 这里选择抛出异常
            raise

        if self.transform:
            image = self.transform(image)

        # 获取属性标签并生成文本描述
        attrs = self.attr_df.iloc[idx][self.attr_names].values
        attrs = (attrs + 1) // 2  # 将-1转为0，1保持1
        attrs = attrs.astype(int)
        active_attrs = [attr for attr, val in zip(self.attr_names, attrs) if val == 1]

        # 手动截断属性数量，例如，最多10个属性，以避免超长文本
        max_attrs = 10
        active_attrs = active_attrs[:max_attrs]

        if len(active_attrs) == 0:
            text_prompt = "A person with no notable attributes."
        else:
            text_description = ', '.join(active_attrs)
            text_prompt = f"A person with {text_description}."

        # 打印前5个样本的文本描述和图像形状
        if idx < 5:
            print(f"Index: {idx}, Text Prompt: {text_prompt}")
            print(f"Image shape: {image.shape}")

        return text_prompt, image

# 初始化 CLIP 模型
clip_model, preprocess_clip = clip.load("ViT-B/32", device=device)
clip_model.eval()

# 创建数据集和数据加载器，并计算嵌入
def compute_and_save_embeddings(partition, output_dir):
    dataset = CelebAEmbeddingsDataset(
        img_dir=IMG_DIR,
        attr_path=ATTR_PATH,
        bbox_path=BBOX_PATH,
        partition_path=PARTITION_PATH,
        transform=transform,
        partition=partition
    )
    dataloader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True
    )

    text_embeddings = []
    image_embeddings = []

    with torch.no_grad():
        for text_prompts, images in tqdm(dataloader, desc=f"Computing Embeddings for partition {partition}"):
            # 生成文本嵌入
            try:
                text_tokens = clip.tokenize(text_prompts, truncate=True).to(device)
            except Exception as e:
                print(f"Error tokenizing text: {e}")
                raise

            try:
                text_emb = clip_model.encode_text(text_tokens).cpu()
            except Exception as e:
                print(f"Error encoding text: {e}")
                raise

            text_embeddings.append(text_emb)

            # 生成图像嵌入
            try:
                images = images.to(device)
                image_emb = clip_model.encode_image(images).cpu()
            except Exception as e:
                print(f"Error encoding images: {e}")
                raise

            image_embeddings.append(image_emb)

    # 合并所有嵌入
    if len(text_embeddings) == 0 or len(image_embeddings) == 0:
        raise RuntimeError("未计算到任何嵌入。请检查数据加载和嵌入计算步骤。")

    text_embeddings = torch.cat(text_embeddings, dim=0)
    image_embeddings = torch.cat(image_embeddings, dim=0)

    # 保存嵌入到文件
    torch.save(text_embeddings, os.path.join(output_dir, f'text_embeddings_partition_{partition}.pt'))
    torch.save(image_embeddings, os.path.join(output_dir, f'image_embeddings_partition_{partition}.pt'))
    print(f"嵌入已保存为 'text_embeddings_partition_{partition}.pt' 和 'image_embeddings_partition_{partition}.pt'")

if __name__ == "__main__":
    # 分别计算并保存每个分区的嵌入
    for partition in [0, 1, 2]:  # 0: train, 1: val, 2: test
        compute_and_save_embeddings(partition, OUTPUT_DIR)


使用设备: cuda
Attributes DataFrame columns: ['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']
Attributes DataFrame head:
      image_id  5_o_Clock_Shadow  Arched_Eyebrows  Attractive  Bags_Under_Eyes  \
0  000001.jpg                -1                1           1               -1   
1  000002.jpg                -1               -1          -1                1   
2  000003.jpg                -1               -1          -1               -1   
3  000004.

Computing Embeddings for partition 0:   0%|          | 0/318 [00:00<?, ?it/s]

Index: 0, Text Prompt: A person with Arched_Eyebrows, Attractive, Brown_Hair, Heavy_Makeup, High_Cheekbones, Mouth_Slightly_Open, No_Beard, Pointy_Nose, Smiling, Straight_Hair.
Image shape: torch.Size([3, 224, 224])
Index: 1, Text Prompt: A person with Bags_Under_Eyes, Big_Nose, Brown_Hair, High_Cheekbones, Mouth_Slightly_Open, No_Beard, Smiling, Young.
Image shape: torch.Size([3, 224, 224])
Index: 2, Text Prompt: A person with Big_Lips, Blurry, Male, Narrow_Eyes, No_Beard, Pointy_Nose, Wavy_Hair, Young.
Image shape: torch.Size([3, 224, 224])
Index: 3, Text Prompt: A person with Attractive, No_Beard, Pointy_Nose, Straight_Hair, Wearing_Earrings, Wearing_Lipstick, Wearing_Necklace, Young.
Image shape: torch.Size([3, 224, 224])
Index: 4, Text Prompt: A person with Arched_Eyebrows, Attractive, Big_Lips, Heavy_Makeup, Narrow_Eyes, No_Beard, Pointy_Nose, Wearing_Lipstick, Young.
Image shape: torch.Size([3, 224, 224])


Computing Embeddings for partition 0: 100%|██████████| 318/318 [02:17<00:00,  2.31it/s]


嵌入已保存为 'text_embeddings_partition_0.pt' 和 'image_embeddings_partition_0.pt'
Attributes DataFrame columns: ['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']
Attributes DataFrame head:
      image_id  5_o_Clock_Shadow  Arched_Eyebrows  Attractive  Bags_Under_Eyes  \
0  000001.jpg                -1                1           1               -1   
1  000002.jpg                -1               -1          -1                1   
2  000003.jpg             

...

嵌入已保存为 'text_embeddings_partition_1.pt' 和 'image_embeddings_partition_1.pt'
Attributes DataFrame columns: ['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive', 'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open', 'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns', 'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace', 'Wearing_Necktie', 'Young']
Attributes DataFrame head:
      image_id  5_o_Clock_Shadow  Arched_Eyebrows  Attractive  Bags_Under_Eyes  \
0  000001.jpg                -1                1           1               -1   
1  000002.jpg                -1               -1          -1                1   
2  000003.jpg             

Computing Embeddings for partition 2:   0%|          | 0/39 [00:00<?, ?it/s]

Index: 0, Text Prompt: A person with Arched_Eyebrows, Bags_Under_Eyes, High_Cheekbones, Mouth_Slightly_Open, No_Beard, Pointy_Nose, Smiling, Wavy_Hair, Wearing_Earrings, Wearing_Hat.
Image shape: torch.Size([3, 224, 224])
Index: 1, Text Prompt: A person with Big_Nose, Black_Hair, Bushy_Eyebrows, High_Cheekbones, Narrow_Eyes, No_Beard, Wavy_Hair, Wearing_Lipstick, Young.
Image shape: torch.Size([3, 224, 224])
Index: 2, Text Prompt: A person with Attractive, Brown_Hair, Heavy_Makeup, High_Cheekbones, Mouth_Slightly_Open, No_Beard, Oval_Face, Smiling, Wavy_Hair, Wearing_Lipstick.
Image shape: torch.Size([3, 224, 224])
Index: 3, Text Prompt: A person with Attractive, Heavy_Makeup, High_Cheekbones, Mouth_Slightly_Open, No_Beard, Oval_Face, Pointy_Nose, Rosy_Cheeks, Smiling, Wearing_Earrings.
Image shape: torch.Size([3, 224, 224])
Index: 4, Text Prompt: A person with Black_Hair, High_Cheekbones, Mouth_Slightly_Open, Narrow_Eyes, No_Beard, Smiling, Young.
Image shape: torch.Size([3, 224, 224]

Computing Embeddings for partition 2: 100%|██████████| 39/39 [00:23<00:00,  1.64it/s]


嵌入已保存为 'text_embeddings_partition_2.pt' 和 'image_embeddings_partition_2.pt'
